# Midterm Project

In [1]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt


import os
import csv
import numpy
import datetime
import certifi
import pandas as pd
import json
import numpy

import pymongo
from pymongo import MongoClient
import sqlalchemy
from sqlalchemy import create_engine

### Loaded Data from CSV Files into MySQL 

In [2]:
# read csv
data_dir = os.path.join(os.getcwd())
data_file = os.path.join(data_dir, 'customers.csv')
data_file2 =  os.path.join(data_dir, 'employees.csv')
data_file3 =  os.path.join(data_dir, 'inventory_transactions.csv')
data_file4 =  os.path.join(data_dir, 'order_details.csv')
data_file5 =  os.path.join(data_dir, 'orders.csv')
data_file6 =  os.path.join(data_dir, 'payment_methods.csv')
data_file7 =  os.path.join(data_dir, 'payments.csv')
data_file8 =  os.path.join(data_dir, 'products.csv')
data_file9 =  os.path.join(data_dir, 'purchase_orders.csv')
data_file10 =  os.path.join(data_dir, 'shipping_methods.csv')
data_file11 =  os.path.join(data_dir, 'suppliers.csv')

df_csv = pd.read_csv(data_file, header=0, index_col=0, encoding='utf-16')
df_csv2 = pd.read_csv(data_file2, header=0, index_col=0, encoding='utf-16')
df_csv3 = pd.read_csv(data_file3, header=0, index_col=0)
df_csv4 = pd.read_csv(data_file4, header=0, index_col=0)
df_csv5 = pd.read_csv(data_file5, header=0, index_col=0)
df_csv6 = pd.read_csv(data_file6, header=0, index_col=0)
df_csv7 = pd.read_csv(data_file7, header=0, index_col=0)
df_csv8 = pd.read_csv(data_file8, header=0, index_col=0)
df_csv9 = pd.read_csv(data_file9, header=0, index_col=0)
df_csv10 = pd.read_csv(data_file10, header=0, index_col=0)
df_csv11 = pd.read_csv(data_file11, header=0, index_col=0)

# drop columns
df_csv8.drop(columns=['Color'], axis=1, inplace=True)


### Connection

In [3]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"
src_dbname = 'midterm'
dst_dbname = 'project1'


mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "midterm"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "arv2vp",
    "password" : "Was4137Jar0300Pogo!",
    "cluster_name" : "midterm",
    "cluster_subnet" : "b84ee9q",
    "cluster_location" : "atlas", # "local"
    "db_name" : "project1"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [4]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Create New Database to work with

In [5]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#### Populate MonogoDB with source data

In [7]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd())

json_files = {"customers":'customers.json',
              "employees":'employees.json',
              "products":'products.json',
              "shipping_methods":'shipping_methods.json'
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)      

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [8]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head(2)

,CustomerID,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued
0,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
1,2,C2,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0


In [9]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "employees"

df_employees = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_employees.head(2)

,EmployeeID,EmployeeName
0,1,E1
1,2,E2


In [10]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "products"

df_products = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_products.head(2)

,ProductID,ProductName,Color,ModelDescription,FabricDescription,Category,Gender,ProductLine,Weight,Size,PackSize,Status,InventoryDate,PurchasePrice
0,1,3-182,,AT,182,Undershirts,Girls' Undershirts,Underwear,822,3,Dozen,In Production,7/10/2003,6.6
1,2,3-183,,AT,183,Undershirts,Girls' Undershirts,Underwear,620,3,Dozen,Out of Production,7/10/2003,5.6


In [11]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "shipping_methods"

df_shipping_methods = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_shipping_methods.head(2)

,ShippingMethodID,ShippingMethod
0,1,Ex Works
1,2,Door to Door Service


#### Lookup the Invoice Date Keys from the Date Dimension Table.
##### Get the Data from the Date Dimension Table.

In [12]:
sql_dim_date = "SELECT date_key, full_date FROM midterm.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


#####  Lookup the Surrogate Primary Key (date_key) that Corresponds to the InventoryDate Column

In [13]:
df_dim_product_date = df_dim_date.rename(columns={"date_key" : "product_date_key", "full_date" : "InventoryDate",})
df_products.InventoryDate = df_products.InventoryDate.astype('datetime64[ns]').dt.date
df_products = pd.merge(df_products, df_dim_product_date, on='InventoryDate', how='left')
df_products.drop(['InventoryDate'], axis=1, inplace=True)
df_products.head(2)

,ProductID,ProductName,Color,ModelDescription,FabricDescription,Category,Gender,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key
0,1,3-182,,AT,182,Undershirts,Girls' Undershirts,Underwear,822,3,Dozen,In Production,6.6,20030710
1,2,3-183,,AT,183,Undershirts,Girls' Undershirts,Underwear,620,3,Dozen,Out of Production,5.6,20030710


#### Perform Any Necessary Transformations to the DataFrames

In [14]:
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))
df_customers.head(2)

,customer_key,CustomerID,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued
0,1,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
1,2,2,C2,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0


In [15]:
df_employees.insert(0, "employee_key", range(1, df_employees.shape[0]+1))
df_employees.head(2)

,employee_key,EmployeeID,EmployeeName
0,1,1,E1
1,2,2,E2


In [16]:
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))
df_products.head(2)

,product_key,ProductID,ProductName,Color,ModelDescription,FabricDescription,Category,Gender,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key
0,1,1,3-182,,AT,182,Undershirts,Girls' Undershirts,Underwear,822,3,Dozen,In Production,6.6,20030710
1,2,2,3-183,,AT,183,Undershirts,Girls' Undershirts,Underwear,620,3,Dozen,Out of Production,5.6,20030710


In [17]:
df_shipping_methods.insert(0, "shipping_methods_key", range(1, df_shipping_methods.shape[0]+1))
df_shipping_methods.head(2)

,shipping_methods_key,ShippingMethodID,ShippingMethod
0,1,1,Ex Works
1,2,2,Door to Door Service


#### Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables


In [18]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [19]:
dataframe = df_employees
table_name = 'dim_employees'
primary_key = 'employee_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [20]:
dataframe = df_products
table_name = 'dim_products'
primary_key = 'product_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [21]:
dataframe = df_shipping_methods
table_name = 'dim_shipping_methods'
primary_key = 'shipping_methods_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### Validate

In [22]:
sql_customers = "SELECT * FROM midterm.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,CustomerID,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued
0,1,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
1,2,2,C2,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0


In [23]:
sql_employees = "SELECT * FROM midterm.dim_employees;"
df_dim_employees = get_sql_dataframe(sql_employees, **mysql_args)
df_dim_employees.head(2)

,employee_key,EmployeeID,EmployeeName
0,1,1,E1
1,2,2,E2


In [24]:
sql_products = "SELECT * FROM midterm.dim_products;"
df_dim_products = get_sql_dataframe(sql_products, **mysql_args)
df_dim_products.head(2)

,product_key,ProductID,ProductName,Color,ModelDescription,FabricDescription,Category,Gender,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key
0,1,1,3-182,,AT,182,Undershirts,Girls' Undershirts,Underwear,822,3,Dozen,In Production,6.6,20030710
1,2,2,3-183,,AT,183,Undershirts,Girls' Undershirts,Underwear,620,3,Dozen,Out of Production,5.6,20030710


In [25]:
sql_shipping_methods = "SELECT * FROM midterm.dim_shipping_methods;"
df_dim_shipping_methods = get_sql_dataframe(sql_shipping_methods, **mysql_args)
df_dim_shipping_methods.head(2)

,shipping_methods_key,ShippingMethodID,ShippingMethod
0,1,1,Ex Works
1,2,2,Door to Door Service


### Create and Populate the New Fact Tables
#### Implement the solution using Pandas DataFrames to craft the table

In [26]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [27]:
sql_orders = "SELECT * FROM midterm.orders;"
df_orders = get_dataframe(user_id, pwd, host_name, src_dbname, sql_orders)
df_orders.head(2)

,OrderID,CustomerID,EmployeeID,ShippingMethodID,OrderDate,ShipDate,FreightCharge
0,2,1,1,1.0,7/10/2003,7/10/2003,0.0
1,4,2,2,1.0,7/11/2003,7/11/2003,0.0


In [28]:
sql_order_details = "SELECT * FROM midterm.order_details;"
df_order_details = get_dataframe(user_id, pwd, host_name, src_dbname, sql_order_details)
df_order_details.head(2)

,OrderDetailID,OrderID,ProductID,QuantitySold,UnitSalesPrice
0,2,2,955,5,7.5
1,3,2,958,5,8.5


#### Join the Orders and OrderDetails DataFrames

In [29]:
df_fact_orders = pd.merge(df_orders, df_order_details, on='OrderID', how = 'right')
df_fact_orders.head(2)

,OrderID,CustomerID,EmployeeID,ShippingMethodID,OrderDate,ShipDate,FreightCharge,OrderDetailID,ProductID,QuantitySold,UnitSalesPrice
0,2,1,1,1.0,7/10/2003,7/10/2003,0.0,2,955,5,7.5
1,2,1,1,1.0,7/10/2003,7/10/2003,0.0,3,958,5,8.5


In [30]:
df_fact_orders.shape

(105757, 11)

###  Lookup the Primary Keys from the Dimension Tables

In [31]:
df_dim_customers = df_dim_customers.rename(columns={"customer_key":"customer_id_key", "customer_id":"customer_id"})
df_fact_orders = pd.merge(df_fact_orders, df_dim_customers, on='CustomerID', how='inner')
df_fact_orders.drop(['CustomerID'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,EmployeeID,ShippingMethodID,OrderDate,ShipDate,FreightCharge,OrderDetailID,ProductID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued
0,2,1,1.0,7/10/2003,7/10/2003,0.0,2,955,5,7.5,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
1,2,1,1.0,7/10/2003,7/10/2003,0.0,3,958,5,8.5,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0


In [32]:
df_dim_employees = df_dim_employees.rename(columns={"employee_key":"employee_id_key", "employee_id":"employee_id"})
df_fact_orders = pd.merge(df_fact_orders, df_dim_employees, on='EmployeeID', how='inner')
df_fact_orders.drop(['EmployeeID'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,ShippingMethodID,OrderDate,ShipDate,FreightCharge,OrderDetailID,ProductID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued,employee_id_key,EmployeeName
0,2,1.0,7/10/2003,7/10/2003,0.0,2,955,5,7.5,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0,1,E1
1,2,1.0,7/10/2003,7/10/2003,0.0,3,958,5,8.5,1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0,1,E1


In [33]:
df_dim_products = df_dim_products.rename(columns={"product_key":"product_id_key", "product_id":"product_id"})
df_fact_orders = pd.merge(df_fact_orders, df_dim_products, on='ProductID', how='inner')
df_fact_orders.drop(['ProductID'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,ShippingMethodID,OrderDate,ShipDate,FreightCharge,OrderDetailID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,...,FabricDescription,Category,Gender,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key
0,2,1.0,7/10/2003,7/10/2003,0.0,2,5,7.5,1,C1,...,29,Drawers,Women's Panties,Underwear,997,XXL,Dozen,In Production,7.15,20030710
1,133,1.0,9/10/2003,9/10/2003,0.0,2721,5,7.5,2,C2,...,29,Drawers,Women's Panties,Underwear,997,XXL,Dozen,In Production,7.15,20030710


In [34]:
df_dim_shipping_methods = df_dim_shipping_methods.rename(columns={"shipping_method_key":"shipping_method_id_key", "shipping_method_id":"shipping_method_id"})
df_fact_orders = pd.merge(df_fact_orders, df_dim_shipping_methods, on='ShippingMethodID', how='inner')
df_fact_orders.drop(['ShippingMethodID'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,OrderDate,ShipDate,FreightCharge,OrderDetailID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,...,Gender,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key,shipping_methods_key,ShippingMethod
0,2,7/10/2003,7/10/2003,0.0,2,5,7.5,1,C1,Moscow,...,Women's Panties,Underwear,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works
1,133,9/10/2003,9/10/2003,0.0,2721,5,7.5,2,C2,Moscow,...,Women's Panties,Underwear,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works


### Date Dimension Conversion

In [35]:
df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "OrderDate"})
df_fact_orders.OrderDate = df_fact_orders.OrderDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_order_date, on='OrderDate', how='left')
df_fact_orders.drop(['OrderDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,ShipDate,FreightCharge,OrderDetailID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,Country,...,ProductLine,Weight,Size,PackSize,Status,PurchasePrice,product_date_key,shipping_methods_key,ShippingMethod,order_date_key
0,2,7/10/2003,0.0,2,5,7.5,1,C1,Moscow,Russian Federation,...,Underwear,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030710
1,133,9/10/2003,0.0,2721,5,7.5,2,C2,Moscow,Russian Federation,...,Underwear,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030910


In [36]:
df_dim_shipped_date = df_dim_date.rename(columns={"date_key" : "shipped_date_key", "full_date" : "ShipDate"})
df_fact_orders.ShipDate = df_fact_orders.ShipDate.astype('datetime64[ns]').dt.date

df_fact_orders = pd.merge(df_fact_orders, df_dim_shipped_date, on='ShipDate', how='left')
df_fact_orders.drop(['ShipDate'], axis=1, inplace=True)
df_fact_orders.head(2)

,OrderID,FreightCharge,OrderDetailID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,Country,PriceCategory,...,Weight,Size,PackSize,Status,PurchasePrice,product_date_key,shipping_methods_key,ShippingMethod,order_date_key,shipped_date_key
0,2,0.0,2,5,7.5,1,C1,Moscow,Russian Federation,1,...,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030710,20030710
1,133,0.0,2721,5,7.5,2,C2,Moscow,Russian Federation,1,...,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030910,20030910


### Insert Fact Order Key

In [37]:
df_fact_orders.insert(0, "fact_order_key", range(1, df_fact_orders.shape[0]+1))
df_fact_orders.head(2)

,fact_order_key,OrderID,FreightCharge,OrderDetailID,QuantitySold,UnitSalesPrice,customer_id_key,CustomerName,Region,Country,...,Weight,Size,PackSize,Status,PurchasePrice,product_date_key,shipping_methods_key,ShippingMethod,order_date_key,shipped_date_key
0,1,2,0.0,2,5,7.5,1,C1,Moscow,Russian Federation,...,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030710,20030710
1,2,133,0.0,2721,5,7.5,2,C2,Moscow,Russian Federation,...,997,XXL,Dozen,In Production,7.15,20030710,1,Ex Works,20030910,20030910


### Send back to SQL

In [38]:
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [39]:
table_name = "fact_orders"
primary_key = "fact_order_key"
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, src_dbname, df_fact_orders, table_name, primary_key, db_operation)

## Select Statement

In [47]:
sql_test = """
SELECT customers.`CustomerName` AS `customer_name`,
    SUM(orders.`QuantitySold`) AS `total_quantity`,
    SUM(orders.`UnitSalesPrice`) AS `total_unit_price`
FROM `{0}`.`fact_orders` AS orders
INNER JOIN `{0}`.`dim_customers` AS customers
ON orders.customer_id_key = customers.CustomerID
GROUP BY customers.`CustomerName`
ORDER BY total_unit_price DESC;
""".format(src_dbname)

df_test = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_test)
df_test.head()

,customer_name,total_quantity,total_unit_price
0,C5,97288.0,37561.70
1,C2,54155.0,28443.70
2,C22,6495.0,24289.05
3,C123,18078.0,23752.40
4,C52,18506.0,21558.20
